In [1]:
import torch
import torch.nn as nn

In [2]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0837],
        [-0.0466]], grad_fn=<AddmmBackward0>)

## 参数访问

In [3]:
print(net)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


In [11]:
print(net[2].state_dict())
print(net[2].state_dict().get("weight"))
print(net[2].bias.data)

OrderedDict([('weight', tensor([[-0.2955,  0.0272,  0.1440, -0.3467, -0.0517,  0.0241, -0.2298, -0.0321]])), ('bias', tensor([-0.0085]))])
tensor([[-0.2955,  0.0272,  0.1440, -0.3467, -0.0517,  0.0241, -0.2298, -0.0321]])
tensor([-0.0085])


### 遍历所有参数

In [12]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [13]:
net.state_dict()['2.bias'].data

tensor([-0.0085])

### 嵌套块


In [16]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [17]:
rgnet[0][1][0].bias.data

tensor([-0.3159,  0.0132,  0.4876,  0.2899, -0.0256, -0.0944,  0.4111,  0.4795])

## 参数初始化


In [19]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01) # _ 下标意味着函数在参数上作替换
        nn.init.zeros_(m.bias)
net.apply(init_normal) # net中所有layer作初始化操作
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0027, -0.0048, -0.0011,  0.0146]), tensor(0.))

In [20]:
def init_constant(m): # 初始化为常数
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]


(tensor([1., 1., 1., 1.]), tensor(0.))

In [21]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  0.0000, -9.9956, -0.0000],
        [-5.1185,  0.0000, -0.0000,  5.2460]], grad_fn=<SliceBackward0>)

In [22]:
net[0].weight.data[:] += 1 # 直接修改
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]


tensor([42.0000,  1.0000, -8.9956,  1.0000])

### 参数绑定
参数绑定有什么作用 ？？
 由于模型参数包含梯度，因此在反向传播期间第二个隐藏层 （即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。

共享参数通常可以节省内存，并在以下方面具有特定的好处：

对于图像识别中的CNN，共享参数使网络能够在图像中的任何地方而不是仅在某个区域中查找给定的功能。
对于RNN，它在序列的各个时间步之间共享参数，因此可以很好地推广到不同序列长度的示例。
对于自动编码器，编码器和解码器共享参数。 在具有线性激活的单层自动编码器中，共享权重会在权重矩阵的不同隐藏层之间强制正交。


In [23]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
